### This is a section of code that goes through a file with all of the nutrient limitations given as output in CESM and picks the most limiting nutrient in each cell. It will run when all the limitation files for a year and PFT are in the same nc file, but could also be used for month to month and altered to fit other needs. 

In [1]:
#all my little modules
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import cartopy.util as util
import matplotlib as mpl
from glob import glob
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
import scipy.stats as stats

mpl.rcParams['figure.figsize'] = [15,6]
mpl.rcParams['figure.dpi'] = 150
mpl.rcParams['savefig.dpi'] = 200

mpl.rcParams['font.size'] = 10
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
mpl.rcParams['lines.linewidth']= 2.0
mpl.rcParams['xtick.labelsize']=16
mpl.rcParams['ytick.labelsize']=16

In [3]:
#assumes all limit data is in one nc file named planktondata
#planktonstring will be 'diat', 'sp', 'diaz', 'cocco'
#this if for diatoms - includes SiO3 - can change code to just not include it for other PFTs, can also add in light for all,
#carbon limitation for coccolithophores
#use for three degree instead of one degree by changing lines 6 to 8 to 384,320 instead of 116, 100

def limiting_factor_diat(planktondata,planktonstring):
    P=planktondata[planktonstring+'_P_lim_Cweight_avg_100m'][:,:]
    N=planktondata[planktonstring+'_N_lim_Cweight_avg_100m']
    S=planktondata[planktonstring+'_SiO3_lim_Cweight_avg_100m'][:,:]
    Fe=planktondata[planktonstring+'_Fe_lim_Cweight_avg_100m'][:,:]
    limits=np.zeros([116,100])
    for lat in range(0,116):
        for lon in range(0,100):
            var = {np.nanmin(P[0,lat,lon]):"P",np.nanmin(S[0,lat,lon]):"S",np.nanmin(Fe[0,lat,lon]):"Fe",np.nanmin(N[0,lat,lon]):"N"}
            min_lim = var.get(min(var))
            if min_lim == 'P':
                limits[lat,lon] =2
            if min_lim=='Fe':
                limits[lat,lon]=3
            if min_lim=='S':
                limits[lat,lon]=1
            if min_lim=='N':
                limits[lat,lon]=4
                
    return limits

In [4]:
#can make a land file to add back in land outlines by using a random year
#change shape to 384, 320 if needed
#make sure to subtract land from limitation files when displaying plots, as seen below
def make_land(year):
    land_data = np.zeros([116,100])
    land_data[:,:] = np.nan
    land_data[(year.KMT)!= 0] = 0 #where the maximum bathymetry depth does not equal zero (land) we write zero
    
    return land_data

In [ ]:
#data
casePath='/glade/campaign/univ/ucnn0026/BGC_spinup/archive/MAA2.1.3_CAM4_POP2_CLM4_RTMEBM_CICE4_SGLC_T31_g37_1850_cocco_ciso_TRENCH_SPINUP_PIavgIC_02_NO3/ocn/hist_1yr_mean/'
caseName='MAA2.1.3_CAM4_POP2_CLM4_RTMEBM_CICE4_SGLC_T31_g37_1850_cocco_ciso_TRENCH_SPINUP_PIavgIC_02_NO3'
yr_10=xr.open_mfdataset(casePath + caseName+'.pop.h.0010.nc')

In [ ]:
#example land 
land=make_land(yr_10)

In [ ]:
#example of making the limitation file - I like to save it as a text file at the end in case something happens with my
#network connection and I lose previously run years, especially with the one degree files, which take a while to run.
#Then you can just load them in again as seen below.
lim_10_sp=limiting_factor_NL(yr_10,'sp')
np.savetxt('lim_10_sp', lim_10_sp)
#works by giving np.savetxt('filenametosave',data)

In [ ]:
#loading text in as arrays
#three degree
lim10_diat_si=np.loadtxt('lim_diat_10')
lim10_diat=np.loadtxt('lim_10_diat')
lim10_sp=np.loadtxt('lim_10_sp')
#one degree
lim20_diat_si=np.loadtxt('lim3_20')
lim20_diat=np.loadtxt('lim2_20')
lim20_sp=np.loadtxt('lim1_20')

In [ ]:
fig,ax=plt.subplots(2,3)
fig.subplots_adjust(wspace=0.1)

#levels seperate each value (one through four) assigned to a specific nutrient limitation
max_level = 4.5
min_level = -0.5
step_level = 1

#notice how I am subtracting land in the plots - gives land outlines, which are absent in the limitation arrays
p0=ax[0,0].contourf(lim10_diat_si-land,levels = np.arange(min_level, max_level + step_level, step_level))
ax[0,0].set_title('3x Diatoms w/ Si')
# plt.colorbar(p0,ax=ax[0,0]) - can use this to check your colors

p1=ax[0,1].contourf(lim10_diat-land,levels = np.arange(min_level, max_level + step_level, step_level))
ax[0,1].set_title('3x Diatoms')

p2=ax[0,2].contourf(lim10_sp-land,levels = np.arange(min_level, max_level + step_level, step_level))
ax[0,2].set_title('3x Small Phytoplankton')

#switching land files for a one degree configuration
p5=ax[1,0].contourf(lim3_13_new-land1,levels = np.arange(min_level, max_level + step_level, step_level))
ax[1,0].set_title('1x Diatoms w/Si')

p6=ax[1,1].contourf(lim2_13_new-land1,levels = np.arange(min_level, max_level + step_level, step_level))
ax[1,1].set_title('1x Diatoms')

ax[1,2].contourf(lim1_13_new-land1,levels = np.arange(min_level, max_level + step_level, step_level))
ax[1,2].set_title('1x Small Phytoplankton')


ax[0,0].axis('off')
ax[0,1].axis('off')
ax[0,2].axis('off')
ax[1,0].axis('off')
ax[1,1].axis('off')
ax[1,2].axis('off')

plt.suptitle('Green=Fe, Lime=N, Teal=P, Blue=Si',size=20) #I like to give the legend in the title
plt.subplots_adjust(top=0.9)